In [1]:
import torch
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertConfig, BertPreTrainedModel, BertForPreTraining, BertForMaskedLM
import torch.nn as nn
from tqdm import tqdm, tqdm_notebook
import os
import re
RUBERT_PATH = '/Users/asselnuraliyeva/Downloads/rubert_cased_L-12_H-768_A-12_pt'
modelpath = os.path.join(RUBERT_PATH,'pytorch_model.bin')

I0323 15:42:52.991456 140735734018944 file_utils.py:38] PyTorch version 1.4.0 available.


In [2]:
os.path.isfile(os.path.join(RUBERT_PATH,'pytorch_model.bin'))

True

In [3]:
tokenizer = BertTokenizer.from_pretrained(RUBERT_PATH, do_lower_case=False)
config = BertConfig.from_json_file(os.path.join(RUBERT_PATH,'bert_config.json'))
model = BertForPreTraining.from_pretrained(modelpath, config=config)
model.eval()
from torch import load
di = load(modelpath)

I0323 15:43:00.092642 140735734018944 tokenization_utils.py:335] Model name '/Users/asselnuraliyeva/Downloads/rubert_cased_L-12_H-768_A-12_pt' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming '/Users/asselnuraliyeva/Downloads/rubert_cased_L-12_H-768_A-12_pt' is a path, a model identifier, or url to a directory containing tokenizer files.
I0323 15:43:00.095082 140735734018944 tokenization_utils.py:364] Didn't find file /Users/asselnuraliyeva/Downloads/rubert_

In [4]:
def lab5(text): 
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    segments_ids[0] = 0
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    predictions = model(tokens_tensor, token_type_ids=segments_tensors)
    array_of_thensors = []
    for i in range(len(tokenized_text)):
        array_of_thensors.append((predictions[0][0][i].detach().numpy()))
       
    mean = np.mean(array_of_thensors, axis = 0)
    return mean

In [5]:
df= pd.DataFrame()
df = pd.read_csv('texts_train.txt', sep="\t", encoding='UTF-8',header=0)
df['score'] = pd.read_csv('scores_train.txt', sep="\t", encoding='UTF-8',header=0)
df.columns = ['text', 'score']
print(df.head())

                                                text  score
0  думал, что будет лучше идея очень интересна - ...      7
1  с творчеством Головачева я познакомился посред...     10
2  то-то я и в большое неудовольствие прочитал "А...      5
3  как мне показалось местами сильно смахивает на...      6
4  от первой части книги просто оторваться не мог...      9


In [6]:
unique_grades = np.sort(df['score'].unique())
print('original range of scores: ',unique_grades)
unique_grades  = [3,7,9]
print('new range of scores: ',unique_grades)

original range of scores:  [ 1  2  3  4  5  6  7  8  9 10]
new range of scores:  [3, 7, 9]


In [7]:
df_new = pd.DataFrame()
comment = []
grade = []

for grades in unique_grades:
    counter = 0
    for i in range(len(df)):
        if (counter<300):
            if (grades == df['score'][i]):
                comment.append(df['text'][i])
                grade.append(df['score'][i])
                counter = counter + 1 
df_new['text'] = comment
df_new['score'] = grade
df_new['score'] = df_new['score'].replace(5, 2)
df_new['score'] = df_new['score'].replace(10, 3)
print('substituted range of scores: ',np.sort(df_new['score'].unique()))
print(df_new.head())

substituted range of scores:  [3 7 9]
                                                text  score
0  у автора куча идей, но кошмарен язык и рваный ...      3
1  После Сиальских хроник ожидал от Пехова чего т...      3
2  Сразу хочу сказать, что Ритуал ? это чисто и и...      3
3  Черезмерно затянуто. Содержание не соответству...      3
4  скучно только для фанатов Перумова и сериала о...      3


In [8]:
df_new.to_csv('updated_ds.csv', sep = '|') #save this dataset to csv

In [9]:
array_of_mean_tensors = []
grade = []
comment = []
counter = 0
for i in range(len(df_new)):
    try: 
        array_of_mean_tensors.append(lab5(df_new['text'][i]))
        grade.append(df_new['score'][i])
        comment.append(df_new['text'][i])
    except:
        counter = counter + 1

In [10]:
df_with_tensors = pd.DataFrame()
df_with_tensors['text'] = comment
df_with_tensors['tensor'] = array_of_mean_tensors
df_with_tensors['score'] = grade
df_with_tensors.to_csv('thensors_300.csv', sep = '|')

In [11]:
X = array_of_mean_tensors
y = df_with_tensors['score']

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)

In [13]:
from sklearn.metrics import accuracy_score as ac
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error as mse
def metrics(y_test, y_pred, name_of_alg): 
    mae = np.mean(abs(y_test - y_pred))
    rmse = (mse(y_test, y_pred))**0.5
    print('\bMetrics and score for ', name_of_alg,':')
    print('Accuracy score from rmse:', 100 - np.mean(100 * (rmse / y_test)))
    print('Accuracy score from mae:', 100 - np.mean(100 * (mae / y_test)))
    print('classification_report: ')
    print(classification_report(y_test, y_pred))
    print('confusion matrix')
    print(confusion_matrix(y_test, y_pred))
    print('root mse')    
    print(rmse)
    print('root mae')    
    print(mae)

In [19]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
knn_y_pred = knn.predict(X_test)

In [21]:
def get_mae(pred, test):
    # Calculate the absolute errors
    errors = abs(pred - test)
    # Print out the mean absolute error (mae)
    print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
    return errors

In [22]:
knn_errors = get_mae(knn_y_pred, y_test)

Mean Absolute Error: 2.22 degrees.


In [24]:
def get_accuracy(errs, test_data):
    # Calculate mean absolute percentage error (MAPE)
    mape = 100 * (errs / test_data)
    # Calculate and display accuracy
    accuracy = 100 - np.mean(mape)
    print('Accuracy:', round(accuracy, 2), '%.')
    return accuracy

In [25]:
get_accuracy(knn_errors, y_test)

Accuracy: 62.77 %.


62.773787354792944

In [33]:
for i in range(len(y_test)):
    index = y_test.index[i]
    print(df_with_tensors['text'][index])
    print('targeted_score: ', str(y[index]))
    print('predicted_score: ', str(y_pred[i]))
    print('-----------------------------')

Отличная книга. Главный герой просто молодец - выпутывается из сложных ситуаций, мастерски владеет древним боевым искусством и никогда не забывает, что человек всю жизнь играет представление для богов. В общем любителям фэнтези читать обязательно.
targeted_score:  9
predicted_score:  8.109289091120948
-----------------------------
Книга о вине, войне и сыре.
targeted_score:  7
predicted_score:  5.899907064767543
-----------------------------
Мост через вечность гораздо лучше)
targeted_score:  7
predicted_score:  8.448890618340345
-----------------------------
Вопрос на засыпку - где вы такой анонс книги нашли? Не об этом она... Какие колеса государственной машины? Очень серьезная фантастика, где автор переплел несколько жанров - здесь и космическая фантастика, здесь и утопия, и психология чистой воды. Злая ирония и сарказм. Вера в человечество, в доброту и любовь. Всем любителям книг, где научная фатнастика переплетается с психологией души человеческой!
targeted_score:  9
predicted_sco

predicted_score:  4.9156548257047135
-----------------------------
